<a href="https://colab.research.google.com/github/crislmfroes/tensorflow-colab/blob/master/salvando_e_carregando_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install h5py pyyaml

In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os

In [4]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
train_labels = train_labels[:10000]
test_labels = test_labels[:10000]

train_images = train_images[:10000].reshape(-1, 28 * 28) / 255.
test_images = test_images[:10000].reshape(-1, 28 * 28) / 255.

In [0]:
def create_model():
  model = keras.Sequential([
      keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
      keras.layers.Dropout(0.2),
      keras.layers.Dense(10, activation=tf.nn.softmax)
  ])
  model.compile(optimizer='adam', loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])
  return model

In [9]:
model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [0]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

In [13]:
cp_callback = keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels), callbacks=[cp_callback])

Train on 10000 samples, validate on 10000 samples
Epoch 1/10
 9792/10000 [============================>.] - ETA: 0s - loss: 0.0722 - acc: 0.9798
Epoch 00001: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
10000/10000 [==============================] - 2s 207us/step - loss: 0.0730 - acc: 0.9795 - val_loss: 0.1544 - val_acc: 0.9536
Epoch 2/10
 9792/10000 [============================>.] - ETA: 0s - loss: 0.0545 - acc: 0.9839
Epoch 00002: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
10000/10000 [==============================] - 2s 208us/step - loss: 0.0541 - acc: 0.9839 - val_loss: 0.1495 - val_acc: 0.9548
Epoch 3/10
 9920/10000 [============================>.] - ETA: 0s - loss: 0.0420 - acc: 0.9880
Epoch 00003: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
10000/10000 [==============================] - 2s 208us/step - loss: 0.0418 - acc: 0.9880 - val_los

In [16]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [18]:
model = create_model()
loss, acc = model.evaluate(test_images, test_labels)
print('Untrained model accuracy: {:5.2f}%'.format(acc * 100))

10000/10000 [==============================] - 1s 72us/step
Untrained model accuracy:  9.36%


In [19]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels)
print('Restored model accuracy: {:5.2f}%'.format(acc * 100))

10000/10000 [==============================] - 1s 60us/step
Restored model accuracy: 96.01%


In [21]:
model.save_weights('./checkpoints/my-checkpoint')

model = create_model()
model.load_weights('./checkpoints/my-checkpoint')
loss, acc = model.evaluate(test_images, test_labels)
print('Restored model accuracy: {:5.2f}%'.format(acc * 100))


Consider using a TensorFlow optimizer from `tf.train`.
10000/10000 [==============================] - 1s 65us/step
Restored model accuracy: 96.01%


In [0]:
model.save('my_model.h5')

In [0]:
new_model = keras.models.load_model('my_model.h5')

In [24]:
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [25]:
loss, acc = new_model.evaluate(test_images, test_labels)
print('Restored model accuracy: {:5.2f}%'.format(acc * 100))

10000/10000 [==============================] - 1s 70us/step
Restored model accuracy: 96.01%


In [26]:
saved_model_path = tf.contrib.saved_model.save_keras_model(model, './saved_models')


Consider using a TensorFlow optimizer from `tf.train`.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved_models/temp-b'1545487446'/saved_model.pb


In [27]:
!ls saved_models

1545487446


In [28]:
new_model = tf.contrib.saved_model.load_keras_model(saved_model_path)
new_model

In [0]:
new_model.compile(optimizer='adam', loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [30]:
loss, acc = new_model.evaluate(test_images, test_labels)
print('Restored model accuracy: {:5.2f}%'.format(acc * 100))

10000/10000 [==============================] - 1s 86us/step
Restored model accuracy: 96.01%


In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.